In [2]:
import osmnx as ox
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
import numpy as np
import random
import math
import scipy.stats as stats

In [13]:
#--- COLLECT DATA FROM OPEN STREET MAPS---

def assign_building_attributes(buildings_gdf):
    #give buildings area_m2 attribute
    buildings_gdf = buildings_gdf.to_crs(epsg=32637)
    buildings_gdf['area_m2'] = buildings_gdf['geometry'].area
    graph_bldg_flat = buildings_gdf.reset_index()
    # print(graph_bldg_flat.index)

    #FOR NOW WE RANDOMLY ASSIGN ATTRIBUTES TO THE CHOSEN NEIGHBOURHOOD IN GAZIANTEP BASED ON ESRM DATA TO CREATE A FAKE BUILDING ATTRIBUTE DATASET
    #START WITH COMPUTING THE DISTRIBUTION OF BUILDING TYPES IN GAZIANTEP BASED ON ESRM DATA
    exposure_res_turkey= pd.read_excel("Exposure_Model_Turkey_Res.xlsx")
    exposure_com_turkey= pd.read_excel("Exposure_Model_Turkey_Com.xlsx")
    exposure_ind_turkey= pd.read_excel("Exposure_Model_Turkey_Ind.xlsx")

    #total number of residential buildings in Gaziantep:
    condition1= exposure_res_turkey['NAME_1']=='Gaziantep'
    gazresbldgs= int(sum(exposure_res_turkey[condition1].BUILDINGS))
    # print (gazresbldgs)

    #total number of commercial buildings in Gaziantep:
    condition2= exposure_com_turkey['NAME_1']=='Gaziantep'
    gazcombldgs= int(sum(exposure_com_turkey[condition2].BUILDINGS))
    # print (gazcombldgs)

    #total number of industrial buildings in Gaziantep:
    condition3= exposure_ind_turkey['NAME_1']=='Gaziantep'
    gazindbldgs= int(sum(exposure_ind_turkey[condition3].BUILDINGS))
    # print (gazindbldgs)

    #percentage of residential, commercial and industrial buildings in Gaziantep:
    totalbldgs= gazresbldgs+gazcombldgs+gazindbldgs
    percent_gazres= gazresbldgs/totalbldgs
    percent_gazcom= gazcombldgs/totalbldgs
    percent_gazind= gazindbldgs/totalbldgs
    # print(percent_gazres, percent_gazcom,percent_gazind)

    #NOW BASED ON THESE PROPORTIONS WE CAN PROPORTIONALLY ASSIGN OUR GAZIANTEP BUILDING STOCK INTO RESIDENTIAL COMMERCIAL AND INDUSTRIAL TYPES
    #CLASS FOR BUILDING AND DEFINE ATTRIBUTES BASED ON THE RELEVANT PARAMETERS FOR BUILDING CODE CLASSIFICATION

    class Gaz_bldg:
        def __init__ (self_Gaz_bldg, serial, occupancytype, footprint, structural_system, lateral_resistance, code_compliance, height, occuday, occunight):
            self_Gaz_bldg.serial= serial  
            self_Gaz_bldg.occu_type= occupancytype  
            self_Gaz_bldg.footprint= footprint
            self_Gaz_bldg.strsys= structural_system
            self_Gaz_bldg.latres= lateral_resistance
            self_Gaz_bldg.codecomp= code_compliance
            self_Gaz_bldg.height= height
            self_Gaz_bldg.occu_day= occuday
            self_Gaz_bldg.occu_night= occunight
        
    #NOW PREPARE ALL THE GEOMETRY OF THE OSM IMPORT AS OBJECTS OF CLASS GAZ_BLDG, 
    # AND ASSIGN THEM ATTRIBTES BASED ON PREVAILING GAZIANTEP PATTERNS FROM XML AND CSV DATA ANALYSIS

    #ARBITRARY FOR NOW:
    #STRUCTURAL SYSTEM FROM HAND CALCS ON THE EXCEL FILE
    occtyp_statistical =['residential', 'commercial', 'industrial'] #occupancy type
    occtyp_weights = [percent_gazres, percent_gazcom, percent_gazind]

    strsys_statistical=['CR','MUR','W','S'] #structural system
    strsys_weights=[0.732,0.267,0.0007,0.0003]

    latres_statistical=['LFINF','LDUAL','LWAL','LFM'] #lateral forces resisting system
    latres_weights_CR=[0.209, 0.208, 0.583, 0.000]
    latres_weights_MUR=[0.0, 0.0, 1.0, 0.0]
    latres_weights_W=[0.0, 0.0, 1.0, 0.0]
    latres_weights_S=[0.0, 0.0, 0.0, 1.0]

    codecomp_statistical=['CDN', 'CDL', 'CDM', 'CDH'] #building code compliance
    codecomp_weights=[0.1, 0.3, 0.4, 0.2] # based on analysis of Gaziantep data- Approximate 

    ht_statistical=[1,2,3,4,5,6,7,8,9,10,11,12]
    ht_weights=[0.2, 0.15, 0.15, 0.1, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05] # based on analysis of Gaziantep data- Approximate 

    bldngs = []
    for i in graph_bldg_flat.index:
        occtyp_random = random.choices(occtyp_statistical,weights=occtyp_weights)[0] # here the res, commm, and ind percentages were computed as 0.9283591028593825 0.0534356187548717 0.018205278385745805        
        strsys_random = random.choices(strsys_statistical,weights=strsys_weights)[0]        
        strsys=strsys_random
        
        latres = None
        if strsys=='CR':
            latres=random.choices(latres_statistical,latres_weights_CR)[0]
        elif strsys=='MUR':
            latres=random.choices(latres_statistical,latres_weights_MUR)[0]
        elif strsys=='W':
            latres=random.choices(latres_statistical,[0,0,0,1])[0]
        elif strsys=="S":
            latres=random.choices(latres_statistical, latres_weights_S)[0]
        else:
            latres = 'stinky whoopsie'
        
        codecomp_random = None
        if strsys=='CR' and latres=='LDUAL':
            codecomp_random=random.choices(codecomp_statistical, [0, 0.3, 0.4, 0.3])[0]
        elif strsys=='CR' and latres == 'LWAL':
            codecomp_random=random.choices(codecomp_statistical, [0, 0.3, 0.4, 0.3])[0]
        elif strsys=='MUR':
            codecomp_random=random.choices(codecomp_statistical, [0, 0, 1, 0])[0]
        elif strsys== 'S' and latres=='LFINF':
            codecomp_random=random.choices(codecomp_statistical, [0, 0, 0.4, 0.6])[0]
        elif strsys== 'S' and latres=='LFM':
            codecomp_random=random.choices(codecomp_statistical, [0, 0, 0.4, 0.6])[0]       
        elif strsys== 'S' and latres=='LWAL':
            codecomp_random=random.choices(codecomp_statistical, [0, 0, 0.4, 0.6])[0]   
        elif strsys== 'W':
            codecomp_random=random.choices(codecomp_statistical, [0, 0.3, 0.4, 0.3])[0]   
        else:   
            codecomp_random= random.choices(codecomp_statistical,codecomp_weights)[0]


        ht_random= None #random.choices(ht_statistical,ht_weights)[0]
        if strsys=='CR' and latres=='LFINF': 
            ht_random= random.choices(ht_statistical,[0.3,0.2,0.2,0.1,0.1,0.1,0,0,0,0,0,0])[0]
        elif strsys=='CR' and latres=='LFM': 
            ht_random= random.choices(ht_statistical,[0.3,0.2,0.2,0.1,0.1,0.1,0,0,0,0,0,0])[0]
        elif strsys== "MUR":
            ht_random= random.choices(ht_statistical,[0.3,0.3,0.2,0.1,0.1,0,0,0,0,0,0,0])[0]
        elif strsys== 'W':
            ht_random= random.choices(ht_statistical,[0.3,0.2,0.2,0.1,0.1,0.1,0,0,0,0,0,0])[0]
        else:
            ht_random=random.choices(ht_statistical,ht_weights)[0]

        occu_day = None
        if occtyp_random=='residential':
            occu_day=0.0171
        elif occtyp_random=='commercial':
            occu_day=0.077
        elif occtyp_random=='industrial':
            occu_day=0.087
        else:
            print('we do not know how many people are in these buildings, call the census takers!')

        occu_night = None
        if occtyp_random=='residential':
            occu_night=0.06
        elif occtyp_random=='commercial':
            occu_night=0.005
        elif occtyp_random=='industrial':
            occu_night=0.006
        else:
            print('we do not know how many people are in these buildings, call the census takers!')   

        #now we have a statistically believable distributon of building objcts and attributes for the buildings, and we can make the set of buildings
        bldg = Gaz_bldg(i,occtyp_random, [buildings_gdf['area_m2'][i]], strsys_random, latres ,codecomp_random,ht_random, occu_day, occu_night)
        bldngs.append(bldg)

    #NOW WE CREATE A DATADRAME OF THE GENERATED SYNTHETIC DATA FOR INSPECTION AND CHECKS
    # 1. We convert the list of objects into a dictionary
    bldgobjects = []
    for bldg in bldngs:
        bldgobjects.append({
            'occupancytype' : bldg.occu_type,
            'Footprint' : bldg.footprint[0],
            'structural_system' : bldg.strsys,
            'lateral_resistance' : bldg.latres,
            'code_compliance' : bldg.codecomp,
            'height' : bldg.height,
            'population day' : bldg.occu_day*bldg.footprint[0]*bldg.height,
            'population night' : bldg.occu_night*bldg.footprint[0]*bldg.height
        })
    # print(bldgobjects)
    
    # 2. we create a new dataframe and export it to xlsx
    bldgobjects_dataframe_flat= pd.DataFrame(bldgobjects)
    # bldgobjects_dataframe_flat.to_excel('bldgobjects_flat.xlsx', index=True)
    
    # # NOW WE COMBINE THE RANDOMLY ATTRIBUTED OBJECTS DATAFRAME, TO THE ORIGINAL OSM DATAFRAME TO CREATE ONE BIG DATAFRAME.

    consolidated_bldgdataframe = graph_bldg_flat.merge(bldgobjects_dataframe_flat, left_index=True, right_index=True)
    
    return consolidated_bldgdataframe, bldngs

In [16]:
graph_bldg = ox.features_from_address('Sarıgüllük Mahallesi, Gaziantep', {'building': True}, dist=400)
consolidated_bldgdataframe, bldngs = assign_building_attributes(graph_bldg)
# print(consolidated_bldgdataframe)
print(bldngs)

[<__main__.assign_building_attributes.<locals>.Gaz_bldg object at 0x000001EDF1A59C10>, <__main__.assign_building_attributes.<locals>.Gaz_bldg object at 0x000001EDF1A59EB0>, <__main__.assign_building_attributes.<locals>.Gaz_bldg object at 0x000001EDF1A599A0>, <__main__.assign_building_attributes.<locals>.Gaz_bldg object at 0x000001EDF1A59E50>, <__main__.assign_building_attributes.<locals>.Gaz_bldg object at 0x000001EDF1A59730>, <__main__.assign_building_attributes.<locals>.Gaz_bldg object at 0x000001EDF1A59190>, <__main__.assign_building_attributes.<locals>.Gaz_bldg object at 0x000001EDF1A59760>, <__main__.assign_building_attributes.<locals>.Gaz_bldg object at 0x000001EDF1A59940>, <__main__.assign_building_attributes.<locals>.Gaz_bldg object at 0x000001EDF1A598E0>, <__main__.assign_building_attributes.<locals>.Gaz_bldg object at 0x000001EDF1A59310>, <__main__.assign_building_attributes.<locals>.Gaz_bldg object at 0x000001EDF1A59E80>, <__main__.assign_building_attributes.<locals>.Gaz_bld

In [17]:
#EARTHQUAKE INPUT DATA
#NOW WE GET TO THE EARTHQUAKE PART: WE MAKE A CLASS- EARTHQUAKES, AND GIVE IT THE RELEVANT ATTRIBUTES


class Earthquake:
    def __init__ (self, accg, pga, sa03, sa06, sa10, time):
        self.accg= accg #acceleration by Gravity
        self.pga= pga
        self.sa03= sa03
        self.sa06= sa06
        self.sa10= sa10
        self.time= time

#we can now input some earthquake presets:

Turkey_Feb2023_Quake= Earthquake(9.8, 0.49, 0.98, 0.98, 1.2, 400)
#can add some more earthquakes here
Fiction_Quake= Earthquake(9.8, 0.49, 0.98, 0.98, 1.2, 1200)
Fiction_Quake2= Earthquake(9.8, 1, 2, 2, 2.4, 200)
equake= Turkey_Feb2023_Quake
print(Fiction_Quake.time)

#have to add some others here



1200


In [24]:
def assign_fragility_attributes(consolidated_bldgdataframe, bldngs, equake):
    # read the fragility table
    medbetatable= pd.read_excel("core- fragility values simplified.xlsx", sheet_name='Final')
    
    im_values = []
    imtype_values =[]
    median_ds2_values = []
    median_ds3_values = []
    median_ds4_values = []
    median_ds5_values = []
    beta_values=[]

    # Assuming bldngs is a list of buildings with attributes
    for bldg in bldngs:
        # Create a mask for the current building
        mask = (medbetatable['strsys'] == bldg.strsys) & \
            (medbetatable['latres'] == bldg.latres) & \
            (medbetatable['codecomp'] == bldg.codecomp) & \
            (medbetatable['height'] == bldg.height)
        
        # Use the mask to get the dominant IM and Median values for the current building
        im= medbetatable.loc[mask,'IMT'].values
        for i in im:
            if i == 'PGA':
                imvar= equake.pga
            elif i== 'SA(0.3)':
                imvar= equake.sa03
            elif i== 'SA(0.6)':
                imvar= equake.sa06
            elif i== 'SA(1.0)':
                imvar= equake.sa10
            else:
                print ('error retrieving intensity measure!')
                
        if len(im) > 0:
            im_values.append(imvar)

        imtype= medbetatable.loc[mask,'IMT'].values
        for i in imtype:
            if i == 'PGA':
                imvartype= 'PGA'
            elif i== 'SA(0.3)':
                imvartype= 'SA(0.3)'
            elif i== 'SA(0.6)':
                imvartype= 'SA(0.6)'
            else:
                imvartype= 'SA(1.0)'
                
        if len(imtype) > 0:
            imtype_values.append(imvartype)

        median_ds2 = medbetatable.loc[mask, 'Median_DS1'].values
        if len(median_ds2) > 0:
            median_ds2_values.append(median_ds2[0])
        median_ds3 = medbetatable.loc[mask, 'Median_DS2'].values
        if len(median_ds3) > 0:
            median_ds3_values.append(median_ds3[0])        
        median_ds4 = medbetatable.loc[mask, 'Median_DS3'].values
        if len(median_ds4) > 0:
            median_ds4_values.append(median_ds4[0])
        median_ds5 = medbetatable.loc[mask, 'Median_DS4'].values
        if len(median_ds5) > 0:
            median_ds5_values.append(median_ds5[0])
        beta = medbetatable.loc[mask, 'Beta'].values
        if len(beta) > 0:
            beta_values.append(beta[0])        

    # now we append the various mediand and beta columns to the geodataframe
    consolidated_bldgdataframe['IM_dom']= (im_values)
    consolidated_bldgdataframe['IMtype']= (imtype_values)
    consolidated_bldgdataframe['median_ds2']= (median_ds2_values)
    consolidated_bldgdataframe['median_ds3']= (median_ds3_values)
    consolidated_bldgdataframe['median_ds4']= (median_ds4_values)
    consolidated_bldgdataframe['median_ds5']= (median_ds5_values)
    consolidated_bldgdataframe['beta']= (beta_values)
    # consolidated_bldgdataframe.to_excel('consolidated_dataframe.xlsx', index=True)


    #now calculate the probability of damage states
    e= 2.71828
    pi= math.pi

    def p_ds2_compute(row):
        val1= row['median_ds2'] # the median value
        mean= np.log(val1) #mean median relation
        val2= row['beta'] # the beta (logstandard deviation)
        IM= row['IM_dom'] #goes in the x axis
        lognorm_dist= stats.lognorm(scale=np.exp(mean), s=val2)

        result= lognorm_dist.pdf(IM)
        return result

    def p_ds3_compute(row):
        val1= row['median_ds3'] # the median value
        mean= np.log(val1) #mean median relation
        val2= row['beta'] # the beta (logstandard deviation)
        IM= row['IM_dom'] #goes in the x axis
        lognorm_dist= stats.lognorm(scale=np.exp(mean), s=val2)

        result= lognorm_dist.pdf(IM)
        return result
    def p_ds4_compute(row):
        val1= row['median_ds4'] # the median value
        mean= np.log(val1) #mean median relation
        val2= row['beta'] # the beta (logstandard deviation)
        IM= row['IM_dom'] #goes in the x axis
        lognorm_dist= stats.lognorm(scale=np.exp(mean), s=val2)

        result= lognorm_dist.pdf(IM)
        return result
    def p_ds5_compute(row):
        val1= row['median_ds5'] # the median value
        mean= np.log(val1) #mean median relation
        val2= row['beta'] # the beta (logstandard deviation)
        IM= row['IM_dom'] #goes in the x axis
        lognorm_dist= stats.lognorm(scale=np.exp(mean), s=val2)

        result= lognorm_dist.pdf(IM)
        return result

    #append results
    consolidated_bldgdataframe['p_ds2'] = consolidated_bldgdataframe.apply(p_ds2_compute, axis =1)
    consolidated_bldgdataframe['p_ds3'] = consolidated_bldgdataframe.apply(p_ds3_compute, axis =1)
    consolidated_bldgdataframe['p_ds4'] = consolidated_bldgdataframe.apply(p_ds4_compute, axis =1)
    consolidated_bldgdataframe['p_ds5'] = consolidated_bldgdataframe.apply(p_ds5_compute, axis =1)

    def p_ds5_collapse_compute(row):
        val1= row['p_ds5']
        result= row['p_ds5']*0.1
        return result

    consolidated_bldgdataframe['p_ds5_collapse'] = consolidated_bldgdataframe.apply(p_ds5_collapse_compute, axis =1)
    # consolidated_bldgdataframe.to_excel('consolidated_dataframe.xlsx', index=True)
    return consolidated_bldgdataframe, medbetatable

In [26]:
consolidated_bldgdataframe, medbetatable = assign_fragility_attributes(consolidated_bldgdataframe, bldngs, equake)
# print(consolidated_bldgdataframe)
print(medbetatable)

             Typology strsys latres codecomp  height      IMT  Median_DS1  \
0     CR_LDUAL-DUH_H1     CR  LDUAL      CDH       1      PGA    1.388093   
1    CR_LDUAL-DUH_H10     CR  LDUAL      CDH      10  SA(0.6)    1.376820   
2    CR_LDUAL-DUH_H11     CR  LDUAL      CDH      11  SA(0.6)    1.625920   
3    CR_LDUAL-DUH_H12     CR  LDUAL      CDH      12  SA(0.6)    1.671852   
4     CR_LDUAL-DUH_H2     CR  LDUAL      CDH       2      PGA    0.729419   
..                ...    ...    ...      ...     ...      ...         ...   
243      W_LFM-DUM_H2      W    LFM      CDM       2  SA(0.3)    1.234578   
244      W_LFM-DUM_H3      W    LFM      CDM       3  SA(0.3)    1.526669   
245      W_LFM-DUM_H4      W    LFM      CDM       4  SA(0.6)    0.972227   
246      W_LFM-DUM_H5      W    LFM      CDM       5  SA(0.6)    1.205610   
247      W_LFM-DUM_H6      W    LFM      CDM       6  SA(0.6)    1.513307   

     Median_DS2  Median_DS3  Median_DS4      Beta  \
0      2.212655    2.7

In [32]:
def assign_injury_numbers(consolidated_bldgdataframe, bldngs, medbetatable, equake):
    #now we calculate the actual injury numbers for each building
    ds1_numbers=[]
    ds2_numbers=[]
    ds3_numbers=[]
    ds4_numbers=[]
    ds5_numbers=[]

    # 
    for bldg in bldngs:
        mask = (medbetatable['strsys'] == bldg.strsys) & \
                (medbetatable['latres'] == bldg.latres) & \
                (medbetatable['codecomp'] == bldg.codecomp) & \
                (medbetatable['height'] == bldg.height)   
        pds1n = [float(x) for x in medbetatable.loc[mask, 'injury severity DS1'].values[0].split(',')]
        pds2n = [float(x) for x in medbetatable.loc[mask, 'injury severity DS2'].values[0].split(',')]
        pds3n = [float(x) for x in medbetatable.loc[mask, 'injury severity DS3'].values[0].split(',')]
        pds4n = [float(x) for x in medbetatable.loc[mask, 'injury severity DS4'].values[0].split(',')]
        pds5n = [float(x) for x in medbetatable.loc[mask, 'injury severity DS5+collapse'].values[0].split(',')]
        ds1_numbers.append([pds1n])
        ds2_numbers.append([pds2n])
        ds3_numbers.append([pds3n])
        ds4_numbers.append([pds4n])
        ds5_numbers.append([pds5n])
    consolidated_bldgdataframe['ds1numbers'] = ds1_numbers
    consolidated_bldgdataframe['ds2numbers'] = ds2_numbers
    consolidated_bldgdataframe['ds3numbers'] = ds3_numbers
    consolidated_bldgdataframe['ds4numbers'] = ds4_numbers
    consolidated_bldgdataframe['ds5numbers'] = ds5_numbers
    # consolidated_bldgdataframe.to_excel('consolidated_dataframe.xlsx', index=True)


    def injury_profile_compute_day(row):
        popday= row['population day']
        popnight= row['population night']
        pds2 = row['p_ds2']
        pds3 = row['p_ds3']
        pds4 = row['p_ds4']
        pds5 = row['p_ds5']
        pds5col = row['p_ds5_collapse']
        p5= random.choices([pds5, pds5col], [9,1])
        
        b= [popday*pds2*i for i in pds2n]
        c= [popday*pds3*i for i in pds3n]
        d= [popday*pds4*i for i in pds4n]
        e= [popday*pds5*i for i in pds5n]

        q= ((b[0]*pds2)+(c[0]*pds3)+(d[0]*pds4)+(e[0]*pds5))/(pds2+pds3+pds4+pds5)
        r= ((b[1]*pds2)+(c[1]*pds3)+(d[1]*pds4)+(e[1]*pds5))/(pds2+pds3+pds4+pds5)
        s= ((b[2]*pds2)+(c[2]*pds3)+(d[2]*pds4)+(e[2]*pds5))/(pds2+pds3+pds4+pds5)
        t= ((b[3]*pds2)+(c[3]*pds3)+(d[3]*pds4)+(e[3]*pds5))/(pds2+pds3+pds4+pds5)
        p= (int(popday)-q-r-s-t)
        result= [p,q,r,s,t]
        resultfiction= [(int(popday)-2*q-2*r-2*s-2*t),2*q,2*r,2*s,2*t] #the result values were unrealistically low, - not meaningful for part B
        return np.round(resultfiction)

    def injury_profile_compute_night(row):
        popday= row['population day']
        popnight= row['population night']
        pds2 = row['p_ds2']
        pds3 = row['p_ds3']
        pds4 = row['p_ds4']
        pds5 = row['p_ds5']
        pds5col = row['p_ds5_collapse']
        p5= random.choices([pds5, pds5col], [9,1])
        
        b= [popnight*pds2*i for i in pds2n]
        c= [popnight*pds3*i for i in pds3n]
        d= [popnight*pds4*i for i in pds4n]
        e= [popnight*pds5*i for i in pds5n]

        q= ((b[0]*pds2)+(c[0]*pds3)+(d[0]*pds4)+(e[0]*pds5))/(pds2+pds3+pds4+pds5)
        r= ((b[1]*pds2)+(c[1]*pds3)+(d[1]*pds4)+(e[1]*pds5))/(pds2+pds3+pds4+pds5)
        s= ((b[2]*pds2)+(c[2]*pds3)+(d[2]*pds4)+(e[2]*pds5))/(pds2+pds3+pds4+pds5)
        t= ((b[3]*pds2)+(c[3]*pds3)+(d[3]*pds4)+(e[3]*pds5))/(pds2+pds3+pds4+pds5)
        p= (int(popnight)-q-r-s-t)
        result2= [p,q,r,s,t]
        resultfiction= [(int(popnight)-2*q-2*r-2*s-2*t),2*q,2*r,2*s,2*t] #the result values were unrealistically low, - not meaningful for part B    
        return np.round(resultfiction)



    if 900 < equake.time < 1800:
        consolidated_bldgdataframe['injuries'] = consolidated_bldgdataframe.apply(injury_profile_compute_day, axis =1)
        print('day')
    elif (0 < equake.time < 899) or (1801 < equake.time < 2359):
        consolidated_bldgdataframe['injuries'] = consolidated_bldgdataframe.apply(injury_profile_compute_night, axis =1)
        print('night')
    else:
        print ('arghhh')

    #final datafraem from part A
    partA_data= consolidated_bldgdataframe
    return partA_data

In [34]:
partA_data = assign_injury_numbers(consolidated_bldgdataframe, bldngs, medbetatable)
print(partA_data)
partA_data.to_excel('consolidated_dataframe.xlsx', index=True)

night
    element_type       osmid  \
0            way   222641301   
1            way   353000676   
2            way   353000677   
3            way   353000678   
4            way   353002782   
..           ...         ...   
382          way  1149192726   
383          way  1149192727   
384          way  1149192728   
385          way  1149192729   
386          way  1149192730   

                                                 nodes             addr:city  \
0    [2315882273, 2315882276, 2315882275, 231588227...  Şehitkamil/Gaziantep   
1    [3587854734, 3587854735, 3587854736, 358785473...                   NaN   
2    [3587854738, 3587854739, 3587854740, 358785474...                   NaN   
3    [3587854742, 3587854743, 3587854744, 358785474...                   NaN   
4    [3587874713, 3587874714, 3587874715, 358787471...                   NaN   
..                                                 ...                   ...   
382  [10691145383, 10691145379, 10691145380, 1069

night


In [ ]:
#APPENDIX OF THINGS

############ start brent method ###########

# gaz_bldg_stock=[]

# for i in graph_bldg.index:
#     ser=i[1]
#     occtyp=occtyp_random
#     footpr=graph_bldg.area[i]
#     strsys=strsys_random
#     if strsys=='CR':
#         latres=random.choices(latres_statistical,latres_weights_CR)
#     elif strsys=='MUR':
#         latres=random.choices(latres_statistical,latres_weights_MUR)
#     elif strsys=='W':
#         latres=random.choices(latres_statistical,latres_weights_W)
#     elif strsys=="S":
#         latres=random.choices(latres_statistical, latres_weights_S)
#     codecomp=codecomp_random
#     ht=ht_random
#     blabla=3000

#     gaz_bldg_instance=Gaz_bldg(ser,occtyp,footpr,strsys,latres,codecomp,ht,blabla)
#     gaz_bldg_stock.append(gaz_bldg_instance)


# print([o.strsys for o in gaz_bldg_stock])


##### end brent method #############
